In [47]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML

# Create input box for user to input a value
input_box = widgets.Text(
    value='',
    placeholder='Type something',
    description='Input:',
    disabled=False
)

# Create a button
button = widgets.Button(
    description='Run Notebook',
    disabled=False,
    button_style='',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me to run the code',
    icon='check'  # (FontAwesome names without the `fa-` prefix)
)

# Create an output widget to capture and display output
output = widgets.Output()

# Define the function to execute when button is clicked
def on_button_click(b):
    with output:
        output.clear_output()  # Clear previous output before displaying new results
        user_input = input_box.value
        print(f'User input is: {user_input}')
        runDiff()

# Link the button to the function
button.on_click(on_button_click)

# Arrange the input_box and button side by side
hbox = widgets.HBox([input_box, button])

# Display the input box, button, and output widget
display(hbox, output)

from IPython.display import display, HTML

def runDiff():
    # Example: Loading two CSV files into DataFrames
    df_old = pd.read_csv('data-sources/row-all-operations.csv')
    df_new = pd.read_csv('data-sources/row-all-operations-diff.csv')
    
    # Step 2: Identify Added, Removed, Changed, and Unchanged Rows
    merged = df_old.merge(df_new, on='id', how='outer', suffixes=('_old', '_new'), indicator=True)
    
    # For "Added" rows (right_only): drop "_old" columns and rename "_new" columns
    added_rows = merged[merged['_merge'] == 'right_only'].copy()
    added_rows = added_rows.drop(columns=[col for col in added_rows.columns if '_old' in col])
    added_rows = added_rows.rename(columns=lambda x: x.replace('_new', ''))
    
    # For "Removed" rows (left_only): drop "_new" columns and rename "_old" columns
    removed_rows = merged[merged['_merge'] == 'left_only'].copy()
    removed_rows = removed_rows.drop(columns=[col for col in removed_rows.columns if '_new' in col])
    removed_rows = removed_rows.rename(columns=lambda x: x.replace('_old', ''))
    
    # Combine the "Added" and "Removed" rows
    combined_rows = pd.concat([added_rows, removed_rows], ignore_index=True)
    
    # Fix the categorical _merge column using rename_categories()
    combined_rows['_merge'] = combined_rows['_merge'].cat.rename_categories({
        'right_only': 'Added', 
        'left_only': 'Removed'
    })
    
    # Rename the _merge column to "Row"
    combined_rows = combined_rows.rename(columns={'_merge': 'Row'})
    
    # Reset index and display the final DataFrame
    combined_rows = combined_rows.reset_index(drop=True)
    
    # List of columns to compare (excluding 'id')
    columns_to_compare = df_old.columns.difference(['id'])
    changed_rows = merged[merged['_merge'] == 'both'].copy()
    
    # Create a boolean column to indicate if the value changed
    for column in columns_to_compare:
        old_col = column + '_old'
        new_col = column + '_new'
        changed_rows['is_changed_' + column] = changed_rows[old_col] != changed_rows[new_col]
    
    # Create a new column 'has_changed' that indicates if any column has changed
    changed_rows['has_changed'] = changed_rows[['is_changed_' + col for col in columns_to_compare]].any(axis=1)
    
    # Filter to only include rows where at least one column has changed
    rows_with_changes = changed_rows[changed_rows['has_changed']].copy()
    
    # Rows that did not change (all 'has_changed' == False)
    unchanged_rows = changed_rows[~changed_rows['has_changed']].copy()
    
    # Drop the 'has_changed' column since it's no longer needed
    rows_with_changes = rows_with_changes.drop(columns=['has_changed'])
    unchanged_rows = unchanged_rows.drop(columns=['has_changed'])

    # Step 3: Display the summary DataFrame
    def display_change_summary():
        # Data for the summary table, including unchanged rows
        change_summary = pd.DataFrame({
            'Change Type': ['Added', 'Removed', 'Changed', 'Unchanged'],
            'Count': [len(added_rows), len(removed_rows), len(rows_with_changes), len(unchanged_rows)]
        })
        
        # Define a function to apply conditional formatting for each row based on 'Change Type'
        def color_summary(val):
            if val == 'Added':
                return 'background-color: #ccffcc; color: black'  # Light green for added
            elif val == 'Removed':
                return 'background-color: #ffcccc; color: black'  # Light red for removed
            elif val == 'Changed':
                return 'background-color: #cce5ff; color: black'  # Light blue for changed
            elif val == 'Unchanged':
                return 'background-color: #f0f0f0; color: black'  # Light gray for unchanged
            return ''  # Default (no color)
        
        # Apply the formatting to the 'Change Type' column using Styler.map
        styled_summary = change_summary.style.map(color_summary, subset=['Change Type'])
        
        # Center align the 'Count' column and give it bold text for better visibility
        styled_summary = styled_summary.set_properties(subset=['Count'], **{'text-align': 'center', 'font-weight': 'bold'})
        
        # Display the final styled summary table
        display(styled_summary)

    display(HTML("<h2>Change Summary</h2>"))
    
    # Call the function to display the formatted summary
    display_change_summary()

    # Step 4: Define a function to apply conditional formatting for combined rows
    def highlight_rows(row):
        if row['Row'] == 'Added':
            return ['background-color: #ccffcc'] * len(row)  # Light green
        elif row['Row'] == 'Removed':
            return ['background-color: #ffcccc'] * len(row)  # Light red
        else:
            return [''] * len(row)
    
    # Apply the formatting function and display the combined DataFrame
    styled_combined_rows = combined_rows.style.apply(highlight_rows, axis=1)
    
    # Display the final combined table with color formatting
    display(HTML("<h2>Combined Added and Removed Rows</h2>"))
    display(styled_combined_rows)
    
    # Reset the index for changed_rows and unchanged_rows
    rows_with_changes = rows_with_changes.reset_index(drop=True)
    unchanged_rows = unchanged_rows.reset_index(drop=True)
    
    # Step 5: Display a header for changed rows
    display(HTML("<h2>Changed Rows</h2>"))
    
    # Step 6: Define a function for conditional formatting of changed rows
    def highlight_specific_changes(row):
        # Prepare the styles for each column
        styles = [''] * len(row)
        for column in columns_to_compare:
            old_col = column + '_old'
            new_col = column + '_new'
            # Only highlight if the value has changed
            if row['is_changed_' + column]:
                styles[row.index.get_loc(old_col)] = 'background-color: #cce5ff'  # Blue for old values
                styles[row.index.get_loc(new_col)] = 'background-color: #FFE5B4'  # Green for new values
        return styles

    # Apply the conditional formatting to changed rows
    styled_changed_rows = rows_with_changes.style.apply(highlight_specific_changes, axis=1)
    
    # Display the final changed rows DataFrame
    display(styled_changed_rows)

    # Step 7: Display a header and the unchanged rows without formatting
    display(HTML("<h2>Unchanged Rows</h2>"))
    display(unchanged_rows)

# Run the diff tool
runDiff()


Output()

,Change Type,Count
0,Added,1
1,Removed,1
2,Changed,2
3,Unchanged,3


,id,species,species_count,Row
0,7,walleye,6.000000,Added
1,5,salmon,1.000000,Removed


,id,species_old,species_count_old,species_new,species_count_new,_merge,is_changed_species,is_changed_species_count
0,2,pollock,5.000000,pollock,3.000000,both,False,True
1,3,salmon,2.000000,halibut,2.000000,both,True,False


,id,species_old,species_count_old,species_new,species_count_new,_merge,is_changed_species,is_changed_species_count
0,1,salmon,1.0,salmon,1.0,both,False,False
1,4,black cod,5.0,black cod,5.0,both,False,False
2,6,sablefish,2.0,sablefish,2.0,both,False,False
